In [1]:
import os
import json
import pandas as pd
import numpy as np
import tqdm
import scipy.sparse as sp

from collections import Counter
from pprint import pprint
from collections import defaultdict

DATA_PATH = 'D:\download'
with open(os.path.join(DATA_PATH, 'test_users.json'), 'r') as f:
    test_users = set(json.load(f)['users'])
    
transactions = pd.read_csv(
    os.path.join(DATA_PATH, 'transactions.csv'),
    dtype={
        'element_uid': np.uint16,
        'user_uid': np.uint32,
        'consumption_mode': 'category',
        'ts': np.float64,
        'watched_time': np.uint64,
        'device_type': np.uint8,
        'device_manufacturer': np.uint8
    })

ser = pd.read_csv("WTforserials(num3).csv")
mm = pd.read_csv("WTformnmov.csv")
OLD = pd.read_csv("WTforfilms(evr0204).csv")
rate=pd.read_csv('ratingsappend(notintransactions).csv')



OLD=OLD.append(ser)
OLD=OLD.append(rate)
OLD=OLD.append(mm)


In [2]:

pd.set_option('display.max_columns', 50)

catal=pd.read_json(os.path.join(DATA_PATH, 'catalogue.json'), orient='index') 
catal['element_uid']=catal.index

catal=catal.reset_index()
catal=catal.drop(['index'], 1)
catal.availability=catal.availability.astype(str)
filmlist=catal[catal.availability == '[]'].element_uid
filmlist=filmlist.reset_index()
filmlist=filmlist.drop(['index'], 1)
for el in filmlist.element_uid:
    catal.availability[catal.element_uid == el] = str(pd.unique(transactions[transactions.element_uid == el].consumption_mode))
dct=    {  '[S]\nCategories (1, object): [S]': "SSS",
         "['purchase', 'rent']": "PR",
         '[], Categories (0, object): []': 'ZER',
         '[S, R]\nCategories (2, object): [S, R]': "RS",
         "['purchase', 'rent', 'subscription']":"PRS",
         "['purchase', 'subscription']": "PS",
         "['subscription']": "SSS",
         "['purchase']": "PPP",
         '[S, R, P]\nCategories (3, object): [S, R, P]': "PRS",
         '[S, P]\nCategories (2, object): [S, P]':  "PS",
         '[P, R]\nCategories (2, object): [P, R]':"PR",
         "['rent']":"RRR",
         "['rent', 'subscription']": "RS",
         '[R, P]\nCategories (2, object): [R, P]': "PR",
         '[S, P, R]\nCategories (3, object): [S, P, R]': "PRS",
         '[P, S]\nCategories (2, object): [P, S]': "PS",
         '[R, S]\nCategories (2, object): [R, S]':"RS",
         '[R]\nCategories (1, object): [R]':"RRR",
         '[P]\nCategories (1, object): [P]': "PPP",
        '0' : 'ZER'}
catal.availability=catal.availability.map(dct)
Counter(catal.availability)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Counter({'SSS': 507,
         'PR': 2660,
         'ZER': 1479,
         'RS': 97,
         'PRS': 4301,
         'PS': 701,
         'PPP': 432,
         'RRR': 23})

In [3]:

transactions=pd.merge(transactions, catal, on=['element_uid'])
del catal
ratings = pd.read_csv(
    os.path.join(DATA_PATH, 'ratings.csv'),
    dtype={
        'element_uid': np.uint16,
        'user_uid': np.uint32,
        'ts': np.float64,
        'rating': np.uint8
    }
)
ratings=ratings.drop(['ts'], 1)
transactions=pd.merge(transactions, ratings, how='left',on=['element_uid', 'user_uid'])
devices=pd.get_dummies(transactions.device_type)
types=pd.get_dummies(transactions.type)
csmode=pd.get_dummies(transactions.consumption_mode)
transactions['duration']=transactions['duration'].map(lambda x: 5 if x == 0 else x)
transactions.watched_time=transactions.watched_time/60
transactions['per']=transactions.watched_time/transactions.duration
transactions=transactions.drop(['device_manufacturer','device_type', 'type', 'consumption_mode'], 1)
transactions=pd.concat([transactions,devices], axis=1)
transactions=pd.concat([transactions,types], axis=1)
transactions=pd.concat([transactions,csmode], axis=1)
del devices, types, csmode
transactions["wt_user_mean"] = transactions.user_uid.map(transactions.groupby("user_uid").watched_time.mean())
transactions["per_user_mean"] = transactions.user_uid.map(transactions.groupby("user_uid").per.mean())
transactions["wt_element_mean"] = transactions.element_uid.map(transactions.groupby("element_uid").watched_time.mean())
transactions["per_elemnt_mean"] = transactions.element_uid.map(transactions.groupby("element_uid").per.mean())

coun = pd.DataFrame.from_dict(Counter(transactions['user_uid']), orient='index')
coun['user_uid']=coun.index
coun['quantity_user_views']=coun[0]

coun=coun.reset_index()

coun=coun.drop(['index'], axis=1)
coun=coun.drop([0],1)
transactions=pd.merge(transactions, coun, on='user_uid')

coun = pd.DataFrame.from_dict(Counter(transactions['element_uid']), orient='index')
coun['element_uid']=coun.index
coun['quantity_element_views']=coun[0]

coun=coun.reset_index()

coun=coun.drop(['index'], axis=1)
coun=coun.drop([0],1)
transactions=pd.merge(transactions, coun, on='element_uid')
del coun
avail=pd.get_dummies(transactions.availability)
transactions=pd.concat([transactions,avail], axis=1)
del avail
transactions=transactions.drop(['availability'],1)

transactions['ts-f1']=transactions.ts- transactions.feature_1
transactions.ts=transactions.ts-min(transactions.ts)+1

transactions['ts-f1']=transactions['ts-f1']/coef
transactions.ts=transactions.ts/coef

transactions=transactions.drop(['attributes'], 1)
transactions.isnull().sum()/transactions.shape[0]

element_uid               0.000000
user_uid                  0.000000
ts                        0.000000
watched_time              0.000000
duration                  0.000000
feature_1                 0.000000
feature_2                 0.000000
feature_3                 0.000000
feature_4                 0.000000
feature_5                 0.000000
rating                    0.962943
per                       0.000000
0                         0.000000
1                         0.000000
2                         0.000000
3                         0.000000
4                         0.000000
5                         0.000000
6                         0.000000
movie                     0.000000
multipart_movie           0.000000
series                    0.000000
P                         0.000000
R                         0.000000
S                         0.000000
wt_user_mean              0.000000
per_user_mean             0.000000
wt_element_mean           0.000000
per_elemnt_mean     

In [4]:
transactionsM=transactions[transactions.movie == 1]
transactionsMM=transactions[transactions.multipart_movie == 1]
transactionsS=transactions[transactions.series == 1]
transactionsMT=transactionsM[~transactionsM.rating.isnull()]
transactionsMF=transactionsM[transactionsM.rating.isnull()]
transactionsMMT=transactionsMM[~transactionsMM.rating.isnull()]
transactionsMMF=transactionsMM[transactionsMM.rating.isnull()]
transactionsST=transactionsS[~transactionsS.rating.isnull()]
transactionsSF=transactionsS[transactionsS.rating.isnull()]

transactionsMT['interaction']=transactionsMT.rating
transactionsMF['interaction']=transactionsMF.per * 10
transactionsMF['interaction']=transactionsMF['interaction'].map(lambda x: 10 if x>10 else x)


transactionsMMT['interaction']=transactionsMMT.rating
transactionsMMF['interaction']=transactionsMMF.per * 4
transactionsMMF['interaction']=transactionsMMF['interaction'].map(lambda x: 10 if x>10 else x)

transactionsST['interaction']=transactionsST.rating
transactionsSF['interaction']=transactionsSF.per * 1.7
transactionsSF['interaction']=transactionsSF['interaction'].map(lambda x: 10 if x>10 else x)

transactions=transactionsMT.append(transactionsMF)
del transactionsMT,transactionsMF
transactions=transactions.append(transactionsMMT)
del transactionsMMT
transactions=transactions.append(transactionsMMF)
del transactionsMMF
transactions=transactions.append(transactionsST)
del transactionsST
transactions=transactions.append(transactionsSF)
del transactionsSF
transactions=transactions.sort_index()

transactions.head()

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pyda

,element_uid,user_uid,ts,watched_time,duration,feature_1,feature_2,feature_3,feature_4,feature_5,rating,per,0,1,2,3,4,5,6,movie,multipart_movie,series,P,R,S,wt_user_mean,per_user_mean,wt_element_mean,per_elemnt_mean,quantity_user_views,quantity_element_views,PPP,PR,PRS,PS,RRR,RS,SSS,ts-f1,interaction
0,3336,5177,184.000071,71.366667,90,4.166108e+07,0.739609,45,1.141929,0.654707,9.0,0.792963,1,0,0,0,0,0,0,1,0,0,0,0,1,76.719048,0.797434,94.06047,1.045116,14,45646,0,0,1,0,0,0,0,188.970839,9.000000
1,3336,349202,183.989107,54.850000,90,4.166108e+07,0.739609,45,1.141929,0.654707,NaN,0.609444,1,0,0,0,0,0,0,1,0,0,0,0,1,55.497619,0.536494,94.06047,1.045116,7,45646,0,0,1,0,0,0,0,188.959875,6.094444
2,3336,117582,183.978661,95.450000,90,4.166108e+07,0.739609,45,1.141929,0.654707,NaN,1.060556,0,0,0,1,0,0,0,1,0,0,0,0,1,42.383333,0.405102,94.06047,1.045116,8,45646,0,0,1,0,0,0,0,188.949429,10.000000
3,3336,523833,183.975545,72.433333,90,4.166108e+07,0.739609,45,1.141929,0.654707,NaN,0.804815,0,0,0,0,0,1,0,1,0,0,0,0,1,62.654762,0.940423,94.06047,1.045116,7,45646,0,0,1,0,0,0,0,188.946313,8.048148
4,3336,571064,183.961773,76.133333,90,4.166108e+07,0.739609,45,1.141929,0.654707,NaN,0.845926,1,0,0,0,0,0,0,1,0,0,0,0,1,27.601852,0.325679,94.06047,1.045116,9,45646,0,0,1,0,0,0,0,188.932541,8.459259


In [5]:

transactions=transactions.drop(['rating', 'watched_time', 'ts', 'element_uid', 'user_uid', 'per'], 1)
transactions['interaction']=transactions['interaction'].map(lambda x: 1 if x>4.999 else 0)

transactions['a']=transactions[1]
transactions['b']=transactions[2]
transactions['c']=transactions[3]
transactions['d']=transactions[4]
transactions['e']=transactions[5]
transactions['f']=transactions[6]
transactions['aa']=transactions[0]


In [6]:
transactions=transactions.drop([0,1,2,3,4,5,6], 1)
transactions.head()

,duration,feature_1,feature_2,feature_3,feature_4,feature_5,movie,multipart_movie,series,P,R,S,wt_user_mean,per_user_mean,wt_element_mean,per_elemnt_mean,quantity_user_views,quantity_element_views,PPP,PR,PRS,PS,RRR,RS,SSS,ts-f1,interaction,a,b,c,d,e,f,aa
0,90,4.166108e+07,0.739609,45,1.141929,0.654707,1,0,0,0,0,1,76.719048,0.797434,94.06047,1.045116,14,45646,0,0,1,0,0,0,0,188.970839,1,0,0,0,0,0,0,1
1,90,4.166108e+07,0.739609,45,1.141929,0.654707,1,0,0,0,0,1,55.497619,0.536494,94.06047,1.045116,7,45646,0,0,1,0,0,0,0,188.959875,1,0,0,0,0,0,0,1
2,90,4.166108e+07,0.739609,45,1.141929,0.654707,1,0,0,0,0,1,42.383333,0.405102,94.06047,1.045116,8,45646,0,0,1,0,0,0,0,188.949429,1,0,0,1,0,0,0,0
3,90,4.166108e+07,0.739609,45,1.141929,0.654707,1,0,0,0,0,1,62.654762,0.940423,94.06047,1.045116,7,45646,0,0,1,0,0,0,0,188.946313,1,0,0,0,0,1,0,0
4,90,4.166108e+07,0.739609,45,1.141929,0.654707,1,0,0,0,0,1,27.601852,0.325679,94.06047,1.045116,9,45646,0,0,1,0,0,0,0,188.932541,1,0,0,0,0,0,0,1


In [7]:
transactions.to_csv("transactions111.csv")

In [8]:
# хар-ки фильмов из транзакций, харк-ки фильмов из катала
#хар-ки юзеров из транзакций
#не забитые рейтинги

In [8]:
del catal, OLD, pred1, ser
del mm, rate, bk

del avail, coun, ratings, devices, types, csmode
del pred, answ
del filmlist



NameError: name 'catal' is not defined

In [10]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

X_train=transactions
X_train=X_train.drop(['interaction'],1)

y_train=transactions.interaction
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9643012 entries, 0 to 9643011
Data columns (total 33 columns):
duration                  int64
feature_1                 float64
feature_2                 float64
feature_3                 int64
feature_4                 float64
feature_5                 float64
movie                     uint8
multipart_movie           uint8
series                    uint8
P                         uint8
R                         uint8
S                         uint8
wt_user_mean              float64
per_user_mean             float64
wt_element_mean           float64
per_elemnt_mean           float64
quantity_user_views       int64
quantity_element_views    int64
PPP                       uint8
PR                        uint8
PRS                       uint8
PS                        uint8
RRR                       uint8
RS                        uint8
SSS                       uint8
ts-f1                     float64
a                         uint8
b    

In [13]:
#del transactions
X_train.duration=X_train.duration.astype(np.int16)
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9643012 entries, 0 to 9643011
Data columns (total 33 columns):
duration                  int16
feature_1                 float64
feature_2                 float64
feature_3                 int64
feature_4                 float64
feature_5                 float64
movie                     uint8
multipart_movie           uint8
series                    uint8
P                         uint8
R                         uint8
S                         uint8
wt_user_mean              float64
per_user_mean             float64
wt_element_mean           float64
per_elemnt_mean           float64
quantity_user_views       int64
quantity_element_views    int64
PPP                       uint8
PR                        uint8
PRS                       uint8
PS                        uint8
RRR                       uint8
RS                        uint8
SSS                       uint8
ts-f1                     float64
a                         uint8
b    

In [16]:

#X_test=t_e[t_e.watched_time.isnull()]
#X_test=X_test.drop([ 'watched_time', 'watched_time_perc'],1)
#y_test = t_e[t_e.watched_time.isnull()].watched_time_perc
print(1)

kf = KFold(n_splits=2, shuffle=True, random_state=241) 
grid=  {'n_estimators': [500,600,700]}

estimator=xgb.XGBClassifier(max_depth=6, n_jobs=-1 )
print(2)
gs = GridSearchCV(estimator, grid, scoring='roc_auc', cv=kf, n_jobs=-1)
gs.fit(X_train, y_train)
Cb=gs.best_params_['n_estimators']
Sb=gs.best_score_
print(Cb, Sb)

#estimator.fit(X_train, y_train)
#submit=estimator.predict_proba(X_test)



1
2
700 0.7797312365585215


In [18]:


grid1=  {'max_depth': [5,7,8]}
estimator2=xgb.XGBClassifier(n_estimators=Cb, n_jobs=-1 )
gs2 = GridSearchCV(estimator2, grid1, scoring='roc_auc', cv=kf, n_jobs=-1)
gs2.fit(X_train, y_train)
Cb2=gs2.best_params_['max_depth']
Sb2=gs2.best_score_
print(Cb2, Sb2)


8 0.7808800720768354


In [19]:
print(3)

3


In [1]:
import os
import json
import pandas as pd
import numpy as np
import tqdm
import scipy.sparse as sp

from collections import Counter
from pprint import pprint
from collections import defaultdict

DATA_PATH = 'D:\download'
with open(os.path.join(DATA_PATH, 'test_users.json'), 'r') as f:
    test_users = set(json.load(f)['users'])
    
transactions = pd.read_csv(
    os.path.join(DATA_PATH, 'transactions.csv'),
    dtype={
        'element_uid': np.uint16,
        'user_uid': np.uint32,
        'consumption_mode': 'category',
        'ts': np.float64,
        'watched_time': np.uint64,
        'device_type': np.uint8,
        'device_manufacturer': np.uint8
    })
with open(os.path.join( 'answ29.03(for2mod-33).json'), 'r') as f:    #вставить сюда из первой модели
    answ = (json.load(f))
    
    
bk=pd.DataFrame.from_dict(answ, orient='index')
ind=[]
for i in range(33):
    ind.append(i)
pred=pd.DataFrame(index=ind, columns=[1, 2])
pred1=pd.DataFrame()
for i in (bk.index):
    pred[1]=i
    for j in range (33):
        pred[2][j]=bk[j][i]
    pred1=pred1.append(pred)
pred1['user_uid']=pred1[1]
pred1['element_uid']=pred1[2]
pred1=pred1.drop([1,2],1)
pred1.head()



pd.set_option('display.max_columns', 50)

catal=pd.read_json(os.path.join(DATA_PATH, 'catalogue.json'), orient='index') 
catal['element_uid']=catal.index

catal=catal.reset_index()
catal=catal.drop(['index'], 1)
catal.availability=catal.availability.astype(str)
filmlist=catal[catal.availability == '[]'].element_uid
filmlist=filmlist.reset_index()
filmlist=filmlist.drop(['index'], 1)
for el in filmlist.element_uid:
    catal.availability[catal.element_uid == el] = str(pd.unique(transactions[transactions.element_uid == el].consumption_mode))
dct=    {  '[S]\nCategories (1, object): [S]': "SSS",
         "['purchase', 'rent']": "PR",
         '[], Categories (0, object): []': 'ZER',
         '[S, R]\nCategories (2, object): [S, R]': "RS",
         "['purchase', 'rent', 'subscription']":"PRS",
         "['purchase', 'subscription']": "PS",
         "['subscription']": "SSS",
         "['purchase']": "PPP",
         '[S, R, P]\nCategories (3, object): [S, R, P]': "PRS",
         '[S, P]\nCategories (2, object): [S, P]':  "PS",
         '[P, R]\nCategories (2, object): [P, R]':"PR",
         "['rent']":"RRR",
         "['rent', 'subscription']": "RS",
         '[R, P]\nCategories (2, object): [R, P]': "PR",
         '[S, P, R]\nCategories (3, object): [S, P, R]': "PRS",
         '[P, S]\nCategories (2, object): [P, S]': "PS",
         '[R, S]\nCategories (2, object): [R, S]':"RS",
         '[R]\nCategories (1, object): [R]':"RRR",
         '[P]\nCategories (1, object): [P]': "PPP",
        '0' : 'ZER'}
catal.availability=catal.availability.map(dct)
Counter(catal.availability)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Counter({'SSS': 507,
         'PR': 2660,
         'ZER': 1479,
         'RS': 97,
         'PRS': 4301,
         'PS': 701,
         'PPP': 432,
         'RRR': 23})

In [2]:
X_test=pd.merge(pred1, catal, on=['element_uid'])
#

X_test['duration']=X_test['duration'].map(lambda x: 5 if x == 0 else x)



In [3]:
coef=(max(transactions.ts)-min(transactions.ts))/184

X_test['ts-f1']=(max(transactions.ts)- X_test.feature_1)/coef

In [4]:
transactions=pd.merge(transactions, catal, on=['element_uid'])
devices=pd.get_dummies(transactions.device_type)
types=pd.get_dummies(transactions.type)
csmode=pd.get_dummies(transactions.consumption_mode)
transactions['duration']=transactions['duration'].map(lambda x: 5 if x == 0 else x)
transactions.watched_time=transactions.watched_time/60
transactions['per']=transactions.watched_time/transactions.duration
transactions=transactions.drop(['device_manufacturer','device_type', 'type', 'consumption_mode'], 1)
transactions=pd.concat([transactions,devices], axis=1)
transactions=pd.concat([transactions,types], axis=1)
transactions=pd.concat([transactions,csmode], axis=1)
transactions["wt_user_mean"] = transactions.user_uid.map(transactions.groupby("user_uid").watched_time.mean())
transactions["per_user_mean"] = transactions.user_uid.map(transactions.groupby("user_uid").per.mean())
transactions["wt_element_mean"] = transactions.element_uid.map(transactions.groupby("element_uid").watched_time.mean())
transactions["per_elemnt_mean"] = transactions.element_uid.map(transactions.groupby("element_uid").per.mean())

coun = pd.DataFrame.from_dict(Counter(transactions['user_uid']), orient='index')
coun['user_uid']=coun.index
coun['quantity_user_views']=coun[0]

coun=coun.reset_index()

coun=coun.drop(['index'], axis=1)
coun=coun.drop([0],1)
transactions=pd.merge(transactions, coun, on='user_uid')

coun = pd.DataFrame.from_dict(Counter(transactions['element_uid']), orient='index')
coun['element_uid']=coun.index
coun['quantity_element_views']=coun[0]

coun=coun.reset_index()

coun=coun.drop(['index'], axis=1)
coun=coun.drop([0],1)
transactions=pd.merge(transactions, coun, on='element_uid')
avail=pd.get_dummies(transactions.availability)
transactions=pd.concat([transactions,avail], axis=1)
transactions=transactions.drop(['availability'],1)

transactions['ts-f1']=transactions.ts- transactions.feature_1
transactions.ts=transactions.ts-min(transactions.ts)+1
coef=(max(transactions.ts)-min(transactions.ts))/184
transactions['ts-f1']=transactions['ts-f1']/coef
transactions.ts=transactions.ts/coef

transactions=transactions.drop(['attributes'], 1)
transactions.isnull().sum()/transactions.shape[0]
transactions['a']=transactions[1]
transactions['b']=transactions[2]
transactions['c']=transactions[3]
transactions['d']=transactions[4]
transactions['e']=transactions[5]
transactions['f']=transactions[6]
transactions['aa']=transactions[0]
transactions=transactions.drop([0,1,2,3,4,5,6], 1)
transactions.head()

,element_uid,user_uid,ts,watched_time,duration,feature_1,feature_2,feature_3,feature_4,feature_5,per,movie,multipart_movie,series,P,R,S,wt_user_mean,per_user_mean,wt_element_mean,per_elemnt_mean,quantity_user_views,quantity_element_views,PPP,PR,PRS,PS,RRR,RS,SSS,ts-f1,a,b,c,d,e,f,aa
0,3336,5177,184.000071,71.366667,90,4.166108e+07,0.739609,45,1.141929,0.654707,0.792963,1,0,0,0,0,1,76.719048,0.797434,94.06047,1.045116,14,45646,0,0,1,0,0,0,0,188.970839,0,0,0,0,0,0,1
1,3336,349202,183.989107,54.850000,90,4.166108e+07,0.739609,45,1.141929,0.654707,0.609444,1,0,0,0,0,1,55.497619,0.536494,94.06047,1.045116,7,45646,0,0,1,0,0,0,0,188.959875,0,0,0,0,0,0,1
2,3336,117582,183.978661,95.450000,90,4.166108e+07,0.739609,45,1.141929,0.654707,1.060556,1,0,0,0,0,1,42.383333,0.405102,94.06047,1.045116,8,45646,0,0,1,0,0,0,0,188.949429,0,0,1,0,0,0,0
3,3336,523833,183.975545,72.433333,90,4.166108e+07,0.739609,45,1.141929,0.654707,0.804815,1,0,0,0,0,1,62.654762,0.940423,94.06047,1.045116,7,45646,0,0,1,0,0,0,0,188.946313,0,0,0,0,1,0,0
4,3336,571064,183.961773,76.133333,90,4.166108e+07,0.739609,45,1.141929,0.654707,0.845926,1,0,0,0,0,1,27.601852,0.325679,94.06047,1.045116,9,45646,0,0,1,0,0,0,0,188.932541,0,0,0,0,0,0,1


In [5]:
del catal
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9643012 entries, 0 to 9643011
Data columns (total 38 columns):
element_uid               uint16
user_uid                  uint32
ts                        float64
watched_time              float64
duration                  int64
feature_1                 float64
feature_2                 float64
feature_3                 int64
feature_4                 float64
feature_5                 float64
per                       float64
movie                     uint8
multipart_movie           uint8
series                    uint8
P                         uint8
R                         uint8
S                         uint8
wt_user_mean              float64
per_user_mean             float64
wt_element_mean           float64
per_elemnt_mean           float64
quantity_user_views       int64
quantity_element_views    int64
PPP                       uint8
PR                        uint8
PRS                       uint8
PS                        uint8

In [7]:
X_test.user_uid=X_test.user_uid.astype(np.int32)
X_test.element_uid=X_test.element_uid.astype(np.int32)

In [8]:
meansu=transactions[['user_uid', 'wt_user_mean', 'quantity_user_views', 'per_user_mean']]
meansu=meansu.drop_duplicates()
X_test=pd.merge(X_test, meansu, on=['user_uid'])
meansf=transactions[['element_uid', 'wt_element_mean', 'quantity_element_views', 'per_elemnt_mean']]
meansf=meansf.drop_duplicates()
X_test=pd.merge(X_test, meansf, on=['element_uid'])
X_test.head()

,user_uid,element_uid,attributes,availability,duration,feature_1,feature_2,feature_3,feature_4,feature_5,type,ts-f1,wt_user_mean,quantity_user_views,per_user_mean,wt_element_mean,quantity_element_views,per_elemnt_mean
0,0,7079,"[5599, 1415, 10128, 473, 7, 336, 52, 29441, 32...",PRS,90,4.175902e+07,0.627208,18,1.141929,0.654707,movie,181.971,72.878694,97,0.811307,77.964312,43872,0.86627
1,131072,7079,"[5599, 1415, 10128, 473, 7, 336, 52, 29441, 32...",PRS,90,4.175902e+07,0.627208,18,1.141929,0.654707,movie,181.971,86.205556,15,0.996590,77.964312,43872,0.86627
2,393222,7079,"[5599, 1415, 10128, 473, 7, 336, 52, 29441, 32...",PRS,90,4.175902e+07,0.627208,18,1.141929,0.654707,movie,181.971,35.671111,15,0.478294,77.964312,43872,0.86627
3,262154,7079,"[5599, 1415, 10128, 473, 7, 336, 52, 29441, 32...",PRS,90,4.175902e+07,0.627208,18,1.141929,0.654707,movie,181.971,70.297436,13,0.781978,77.964312,43872,0.86627
4,13,7079,"[5599, 1415, 10128, 473, 7, 336, 52, 29441, 32...",PRS,90,4.175902e+07,0.627208,18,1.141929,0.654707,movie,181.971,56.680000,5,0.504706,77.964312,43872,0.86627


In [9]:
types=pd.get_dummies(X_test.type)

X_test=pd.concat([X_test,types], axis=1)
X_test=X_test.drop(['attributes', ], 1)

In [10]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1622631 entries, 0 to 1622630
Data columns (total 20 columns):
user_uid                  1622631 non-null int32
element_uid               1622631 non-null int32
availability              1622631 non-null object
duration                  1622631 non-null int64
feature_1                 1622631 non-null float64
feature_2                 1622631 non-null float64
feature_3                 1622631 non-null int64
feature_4                 1622631 non-null float64
feature_5                 1622631 non-null float64
type                      1622631 non-null object
ts-f1                     1622631 non-null float64
wt_user_mean              1622631 non-null float64
quantity_user_views       1622631 non-null int64
per_user_mean             1622631 non-null float64
wt_element_mean           1622631 non-null float64
quantity_element_views    1622631 non-null int64
per_elemnt_mean           1622631 non-null float64
movie                     1622631 

In [11]:
X_test.to_csv('transactions222.csv')